In [4]:
from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  # C binary format

dog = model['dog']
print(dog.shape)
print(dog[:10])

# Deal with an out of dictionary word: ?????? (Michail)
if '??????' in model:
    print(model['??????'].shape)
else:
    print('{0} is an out of dictionary word'.format('??????'))

# Some predefined functions that show content related information for given words
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

print(model.doesnt_match("breakfast cereal dinner lunch".split()))

print(model.similarity('woman', 'man'))

(300,)
[ 0.05126953 -0.02233887 -0.17285156  0.16113281 -0.08447266  0.05737305
  0.05859375 -0.08251953 -0.01538086 -0.06347656]
?????? is an out of dictionary word
[(u'queen', 0.7118192315101624), (u'monarch', 0.6189675331115723), (u'princess', 0.5902431607246399), (u'crown_prince', 0.549946129322052), (u'prince', 0.5377321839332581), (u'kings', 0.5236844420433044), (u'Queen_Consort', 0.5235945582389832), (u'queens', 0.5181134343147278), (u'sultan', 0.5098592638969421), (u'monarchy', 0.5087412595748901)]
cereal
0.7664012231


In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

train_cands =  session.query(Spouse).filter(Spouse.split == 0)

print("Number of candidates:", session.query(Spouse).filter(Spouse.split == 1).count())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
('Number of candidates:', 2796)


In [38]:
import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)
import numpy as np
import gensim.matutils as gm
import re

#spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
spouses = {'spouse', 'wife', 'husband'}
family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
              'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
family = family | {f + '-in-law' for f in family}
other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}
stoplist = ['a','an','the','on']

from gensim.parsing.preprocessing import STOPWORDS

# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    return 0.9 if len(spouses.intersection(get_between_tokens(c))) > 0 else 0.1

def LF_vector_bw(c):
    #print(list(get_between_tokens(c)))
    word_vectors= []
    btw_words = [word for word in get_between_tokens(c) if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalnum()]
    print(' '.join(btw_words))
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            pass
        
    word_vectors = np.array(word_vectors)
    word_vectors = word_vectors.T
    #word_vectors = np.array(word_vectors)
    sent_vec = np.mean(word_vectors, axis=1)
    #print(len(word_vectors),len(word_vectors[0]))
    #print("sent",sent_vec.shape)
    sim = []
    for s in spouses:
        print(s,gensim.matutils.cossim(gm.any2sparse(sent_vec, eps=1e-09),gm.any2sparse(model[s],eps=1e-09)))

#c = train_cands[5]
#LF_vector_bw(c)

for c in train_cands:
    LF_vector_bw(c)
    #print
    
    

came journalist called
('spouse', 0.091736227823886698)
('husband', 0.26686390055543752)
('wife', 0.24426768686649553)

following performance gush thank heart emoji honor globalcitizenfestival actress freida pinto spotted walking carpet
('spouse', 0.16476968074891207)
('husband', 0.3100173963186732)
('wife', 0.29892373921388182)

time singer looked stylish black white striped jumpsuit plunging cut outs grateful scream queens star took
('spouse', 0.095018498342077207)
('husband', 0.25013727531704644)
('wife', 0.2365657204828491)

time singer looked stylish black white striped jumpsuit plunging cut outs grateful scream queens star took twitter following performance gush thank heart emoji honor globalcitizenfestival actress freida pinto spotted walking carpet
('spouse', 0.14369528558822187)
('husband', 0.30832607760643688)
('wife', 0.30125034012469809)

arrived join coldplay duet chic time singer looked stylish black white striped jumpsuit plunging cut outs grateful scream queens star too

IndexError: tuple index out of range

In [1]:
from gensim.parsing.preprocessing import STOPWORDS
print(STOPWORDS)

frozenset(['all', 'six', 'just', 'less', 'being', 'indeed', 'over', 'move', 'anyway', 'four', 'not', 'own', 'through', 'using', 'fify', 'where', 'mill', 'only', 'find', 'before', 'one', 'whose', 'system', 'how', 'somewhere', 'much', 'thick', 'show', 'had', 'enough', 'should', 'to', 'must', 'whom', 'seeming', 'yourselves', 'under', 'ours', 'two', 'has', 'might', 'thereafter', 'latterly', 'do', 'them', 'his', 'around', 'than', 'get', 'very', 'de', 'none', 'cannot', 'every', 'un', 'they', 'front', 'during', 'thus', 'now', 'him', 'nor', 'name', 'regarding', 'several', 'hereafter', 'did', 'always', 'who', 'didn', 'whither', 'this', 'someone', 'either', 'each', 'become', 'thereupon', 'sometime', 'side', 'towards', 'therein', 'twelve', 'because', 'often', 'ten', 'our', 'doing', 'km', 'eg', 'some', 'back', 'used', 'up', 'go', 'namely', 'computer', 'are', 'further', 'beyond', 'ourselves', 'yet', 'out', 'even', 'will', 'what', 'still', 'for', 'bottom', 'mine', 'since', 'please', 'forty', 'per', 